# API Exercises

Here are two exercises that involve working with APIs and dictionaries.

One is using the Open Brewery API found at https://www.openbrewerydb.org/, and the other is using the API for UK Police Data, found at https://data.police.uk/docs/.

You can complete them in either order!

Remember that you can create new cells with esc + a or b

## Breweries

### Q1: Load the first page of results with 50 results per page

In [2]:
url = 'https://api.openbrewerydb.org/breweries?page=1&per_page=50'
import json
import requests

data = requests.get(url)
print(data.status_code)
data.json()

200


[{'id': 2,
  'name': 'Avondale Brewing Co',
  'brewery_type': 'micro',
  'street': '201 41st St S',
  'city': 'Birmingham',
  'state': 'Alabama',
  'postal_code': '35222-1932',
  'country': 'United States',
  'longitude': '-86.774322',
  'latitude': '33.524521',
  'phone': '2057775456',
  'website_url': 'http://www.avondalebrewing.com',
  'updated_at': '2018-08-23T23:19:57.825Z',
  'tag_list': []},
 {'id': 4,
  'name': 'Band of Brothers Brewing Company',
  'brewery_type': 'micro',
  'street': '1605 23rd Ave',
  'city': 'Tuscaloosa',
  'state': 'Alabama',
  'postal_code': '35401-4653',
  'country': 'United States',
  'longitude': '-87.5621551272424',
  'latitude': '33.1984907123707',
  'phone': '2052665137',
  'website_url': 'http://www.bandofbrosbrewing.com',
  'updated_at': '2018-08-23T23:19:59.462Z',
  'tag_list': []},
 {'id': 44,
  'name': 'Trim Tab Brewing',
  'brewery_type': 'micro',
  'street': '2721 5th Ave S',
  'city': 'Birmingham',
  'state': 'Alabama',
  'postal_code': '3523

### Q2: This is only the first 50 results.  Get the next 50 and put them together.

In [3]:
url2 = 'https://api.openbrewerydb.org/breweries?page=2&per_page=50'
data2 = requests.get(url2)
brew_data = data.json() + data2.json()
print(brew_data[:2])


[{'id': 2, 'name': 'Avondale Brewing Co', 'brewery_type': 'micro', 'street': '201 41st St S', 'city': 'Birmingham', 'state': 'Alabama', 'postal_code': '35222-1932', 'country': 'United States', 'longitude': '-86.774322', 'latitude': '33.524521', 'phone': '2057775456', 'website_url': 'http://www.avondalebrewing.com', 'updated_at': '2018-08-23T23:19:57.825Z', 'tag_list': []}, {'id': 4, 'name': 'Band of Brothers Brewing Company', 'brewery_type': 'micro', 'street': '1605 23rd Ave', 'city': 'Tuscaloosa', 'state': 'Alabama', 'postal_code': '35401-4653', 'country': 'United States', 'longitude': '-87.5621551272424', 'latitude': '33.1984907123707', 'phone': '2052665137', 'website_url': 'http://www.bandofbrosbrewing.com', 'updated_at': '2018-08-23T23:19:59.462Z', 'tag_list': []}]


### Q3: How many of these 100 breweries in are in Alaska?

In [4]:
import pandas as pd
df = pd.DataFrame(brew_data)
df_Alaska = df[df['state'] == 'Alaska']
print(df_Alaska.shape)

(3, 14)


### Q4: Of these 100 breweries, what are the different unique brewery types?

In [5]:
df['brewery_type'].unique()

array(['micro', 'contract', 'brewpub', 'regional', 'planning',
       'proprietor'], dtype=object)

### Q5: What is the cloest brewery to "Devil's Potion Brewing Company LLC" ?
* Hint 1: Use Euclidian distance w/ longitude and latitude (assume longitude and latitude are a Carteisan coordinate system)
* Hint 2: You'll have to ignore the entries with "none" for latitude or longitude

In [6]:
!pip install geopy

    100% |████████████████████████████████| 102kB 30.4MB/s a 0:00:01
  Running setup.py bdist_wheel for geographiclib ... done
  Stored in directory: /home/dynamic-monitor-5036/.cache/pip/wheels/99/45/d1/14954797e2a976083182c2e7da9b4e924509e59b6e5c661061
Successfully built geographiclib
smart-open 1.8.0 requires bz2file, which is not installed.
nltk 3.4 requires singledispatch, which is not installed.
You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [7]:
from geopy import distance
import numpy as np
import math

devlat = float(df[df['name'].str.contains('Devil\'s Potion Brewing Company LLC') == True]['latitude'].values[0])
devlong = float(df[df['name'].str.contains('Devil\'s Potion Brewing Company LLC') == True]['longitude'].values[0])

coords_1 = (devlat, devlong)
# coords_2 = (52.406374, 16.9251681)

def distance_calc(row): 
    #start = (row['start_latitute'], row['start_longitude'])
    start = coords_1
    if pd.isna(row['latitude']) == False and pd.isna(row['longitude']) == False:
        stop = (float(row['latitude']), float(row['longitude'])) 
        return distance.distance(start, stop).miles
    else :
      return np.nan

df['distance'] = df.apply(lambda row: distance_calc(row), axis = 1)

print(df[df['distance'] > 0].min()['name'])


1912 Brewing


### Q6: Write a function to find the closest brewery to any other given brewery

In [13]:
#Pass brewery_name
def distance_calc(row): 
    #start = (row['start_latitute'], row['start_longitude'])
    start = coords_1
    if pd.isna(row['latitude']) == False and pd.isna(row['longitude']) == False:
        stop = (float(row['latitude']), float(row['longitude'])) 
        return distance.distance(start, stop).miles
    else :
      return np.nan

def nearest_brewery(bname):
    devlat = float(df[df['name'].str.contains(bname) == True]['latitude'].values[0])
    devlong = float(df[df['name'].str.contains(bname) == True]['longitude'].values[0])
    coords_1 = (devlat, devlong)
    #coords_2 = (52.406374, 16.9251681)
    #df['distance'] = df.update(lambda row: distance_calc(row), axis = 1)
    #df.set_index('id', inplace = True)
    df['distance'] = df['distance'].map(lambda row: distance_calc(row), df.distance)
    return df[df['distance'] > 0].min()['name']

#nearest_brewery('Avondale Brewing Co')
nearest_brewery('Band of Brothers Brewing Company')
#Return closes brewery_name

/opt/conda/envs/learn-env/lib/python3.6/site-packages/pandas/core/ops.py:1167: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


TypeError: invalid type comparison

### Q7: How would you get the first 10 pages from this API and put them all together using a for loop?

# Crime in the UK

### We will be analyzing different crimes reported in the UK as provided by https://data.police.uk/docs/

# Exploratory analysis
##### 1. How many total crimes were there at latitude : 52.63902 and -1.131321 on November of 2017.
Use the street level crimes data, the documentation for the API can be found at https://data.police.uk/docs/method/crime-street/

##### 2. We've queried the API once, but it could get annoying to retype the url over and over again, create a function `make_api_request` that enables you to query the API.


 The parameters for the function should be:
* lat (float) : latitude
* lng (float) : longitude
* date (string): Date in the format YYYY-MM
    * default value = `None`
    
And it should return a json object of 

for more information on default values check out http://blog.thedigitalcatonline.com/blog/2015/02/11/default-arguments-in-python/

##### 3. Write a function `categories_of_crime` that will determine the count of each type of crime for a given latitude and longitude. This is labelled as 'category' in the records. Your function should call the `make_api_request` function you created.

The parameters for the function should be:

* lat (float) : latitude
* lng (float) : longitude
* date (str) default = None

The function should return:
* a dictionary with the count of each type of crime



Once you've created the function, try it with these locations
* lat, lng of 51.5017861,-0.1432319   (Buckingham Palace)
* lat, lng of 53.480161, -2.245163     (Manchester)

**Bonus**: 
* Write a function that determines the difference between Buckingham Palace and Manchester in terms of the number of crimes in each category.
    * In which category is there the largest absolute difference between the category of crime?
* Create a histogram depiction of the categories of crime

##### 4. Create a function `find_outcome_statuses` that will determine outcome statuses for a given latitude and longitude and date (optional)?
Investigate the data to determine where the outcome statuses are located.

**NOTE**: You'll notice that some of these crimes do not have crime outcomes. Make these into the category of "Not Resolved."

**NOTE 2**: These might take a long time to execute if you do not specify a month

**Bonus**: What is the ratio of crimes investigated to those not investigated? Is it higher near London or Manchester?

##### 5. Write a function `month_highest_crimes` that will return the month that had the highest number of crimes for a latitude, longitude and a year.

Inputs
* lat (float) : latitude
* lng (float) : longitude
* year (str) : in the format YYYY

Output
* month with highest crime (int)

**Bonus** Make a graph of how the number of crimes changed over time for a year. This will likely require a new function. Is seasonality a factor? Do the type of crimes change over time?

### Bonus Open Ended Questions

1. Take a look at the https://data.police.uk/docs/method/stops-street/ API. Is there a correlation between gender and being stopped and searched? How about race and being stopped and searched?